# API

## Basics
For reading ventilator data:

In [1]:
from io import open

from ventmap.raw_utils import extract_raw

# create generator that will iterate through file. Specify False to ensure that
# breaths without BS/BE markers will be dropped. If you say True, then breaths
# without BS/BE will be kept
generator = extract_raw(open(<filepath to vent data>), False)
for breath in generator:
    # breath data is output in dictionary format
    flow, pressure = breath['flow'], breath['pressure']


If you want to preprocess a breath file for later usage, or if you intend to process it again then it is suggested to use the `process_breath_file` method

In [2]:
from io import open

from ventmap.raw_utils import process_breath_file, read_processed_file

# This function will output 2 files. The first will just contain raw breath data
# the other will contain higher level processed data
process_breath_file(open(<filepath to vent data>), False, 'new_filename')
raw_filepath_name = 'new_filename.raw.npy'
processed_filepath_name = 'new_filename.processed.npy'

for breath in read_processed_file(raw_filepath_name, processed_filepath_name):
    # breath data is output in dictionary format
    flow, pressure = breath['flow'], breath['pressure']


In [3]:
breath.keys()

dict_keys(['rel_bn', 'vent_bn', 'flow', 'pressure', 'abs_bs', 'bs_time', 'frame_dur', 'dt'])

For extracting metadata (I-Time, TVe, TVi) from files.

In [4]:
from ventmap.breath_meta import get_file_breath_meta

# Data output is normally in list format. Ordering information can be found in
# ventmap.constants.META_HEADER.
breath_meta = get_file_breath_meta(<filepath to vent data>)
# If you want a pandas DataFrame then you can set the optional argument to_data_frame=True
breath_meta = get_file_breath_meta(<filepath to vent data>, to_data_frame=True)


Take a look at the `metadata` dataframe

In [5]:
breath_meta.head()

,BN,ventBN,BS,IEnd,BE,I:E ratio,iTime,eTime,inst_RR,tvi,...,tvi2,tve2,x0_index,abs_time_at_BS,abs_time_at_x0,abs_time_at_BE,rel_time_at_BS,rel_time_at_x0,rel_time_at_BE,min_pressure
0,1,53946,0.02,2.52,9.40,0.36232,2.50,6.90,6.382979,2129.422222,...,2129.422222,2238.049167,125,2016-02-17 08-38-13.587473,2016-02-17 08-38-16.087473,2016-02-17 08-38-22.967473,0.02,2.52,9.40,8.81
1,2,53947,9.42,11.72,18.36,0.34535,2.30,6.66,6.696429,2261.646667,...,2261.646667,2149.061944,115,2016-02-17 08-38-22.987473,2016-02-17 08-38-25.287473,2016-02-17 08-38-31.927473,9.42,11.72,18.36,9.53
2,3,53948,18.38,20.78,27.98,0.33241,2.40,7.22,6.237006,744.514444,...,744.514444,562.745556,120,2016-02-17 08-38-31.947473,2016-02-17 08-38-34.347473,2016-02-17 08-38-41.547473,18.38,20.78,27.98,10.08
3,4,53949,28.00,29.34,30.82,0.89333,1.34,1.50,21.126761,402.528889,...,402.528889,194.674444,67,2016-02-17 08-38-41.567473,2016-02-17 08-38-42.907473,2016-02-17 08-38-44.387473,28.00,29.34,30.82,9.81
4,5,53950,30.84,31.80,32.96,0.81356,0.96,1.18,28.037383,383.351111,...,383.351111,317.724167,48,2016-02-17 08-38-44.407473,2016-02-17 08-38-45.367473,2016-02-17 08-38-46.527473,30.84,31.80,32.96,9.56


For extracting Experimental Breath Metadata from files, which have some pieces of more experimental information that have been developed since 2017 paper.

In [6]:
from ventmap.breath_meta import get_file_experimental_breath_meta

# Data output is normally in list format. Ordering information can be found in
# ventmap.constants.META_HEADER.
exp_breath_meta = get_file_experimental_breath_meta(<filepath to vent data>)
# If you want a pandas DataFrame then you can set the optional argument to_data_frame=True
exp_breath_meta = get_file_experimental_breath_meta(<filepath to vent data>, to_data_frame=True)

Take a look at the Experimental Breath Metadata dataframe

In [7]:
exp_breath_meta.head()

,BN,ventBN,BS,IEnd,BE,I:E ratio,iTime,eTime,inst_RR,tvi,...,dyn_compliance,vol_at_.5_sec,vol_at_.76_sec,vol_at_1_sec,pressure_itime_4,pressure_itime_5,pressure_itime_6,pressure_itime_by_pip5,pressure_itime_by_pip6,pressure_itime_from_front
0,1,53946,0.02,2.52,9.40,0.36232,2.50,6.90,6.382979,2129.422222,...,0.218268,-510.342222,-882.069167,-1095.104722,3.36,3.20,3.14,3.34,3.38,2.56
1,2,53947,9.42,11.72,18.36,0.34535,2.30,6.66,6.696429,2261.646667,...,0.212441,-639.298889,-958.682778,-1147.471667,3.08,2.90,2.86,2.88,3.02,2.36
2,3,53948,18.38,20.78,27.98,0.33241,2.40,7.22,6.237006,744.514444,...,0.081617,-186.508889,-257.480278,-305.858056,2.44,2.42,2.42,2.44,2.46,2.46
3,4,53949,28.00,29.34,30.82,0.89333,1.34,1.50,21.126761,402.528889,...,0.044885,-169.228889,-198.612222,-200.667500,1.38,1.36,1.36,1.38,1.44,1.40
4,5,53950,30.84,31.80,32.96,0.81356,0.96,1.18,28.037383,383.351111,...,0.043672,-195.342222,-271.551111,-316.975556,0.98,0.98,0.96,1.00,1.12,1.02


In [8]:
exp_breath_meta.shape

(96, 49)

## Adding new features - example

The information you can use from raw data file are:

In [9]:
breath.keys()


dict_keys(['rel_bn', 'vent_bn', 'flow', 'pressure', 'abs_bs', 'bs_time', 'frame_dur', 'dt'])

If you want to create a new feature, first define your feature as below:
For example, if you want to calculate the time between the Maximum of flow and the Minimun of flow for each breath.

In [11]:
import pandas as pd
def time_btw_max_min_flow(breath):
    flow = breath['flow']
    dt = breath['dt']
    btw_time = abs((flow.index(max(flow)) - flow.index(min(flow)))*dt)
    return round(btw_time,2)



And then add you function at here, implement `extract_new_feature`, return a list `feature_values` for your new feature.

In [14]:
def extract_new_feature(file):
    feature_values = []
    for breath in extract_raw(file, True):
        #TODO: add new feature function
        btw_time = time_btw_max_min_flow(breath)
        feature_values.append(btw_time)
    return feature_values

btw_times = extract_new_feature(open(<filepath to vent data>))

Finally, add this new feature to your original dataframe.

In [16]:
exp_breath_meta['time_btw_max_min'] = btw_times

Take a look at the dataframe `exp_breath_meta` having one more feature `time_btw_max_min` at the end.

In [17]:
exp_breath_meta.head()

,BN,ventBN,BS,IEnd,BE,I:E ratio,iTime,eTime,inst_RR,tvi,...,vol_at_.5_sec,vol_at_.76_sec,vol_at_1_sec,pressure_itime_4,pressure_itime_5,pressure_itime_6,pressure_itime_by_pip5,pressure_itime_by_pip6,pressure_itime_from_front,time_btw_max_min
0,1,53946,0.02,2.52,9.40,0.36232,2.50,6.90,6.382979,2129.422222,...,-510.342222,-882.069167,-1095.104722,3.36,3.20,3.14,3.34,3.38,2.56,1.88
1,2,53947,9.42,11.72,18.36,0.34535,2.30,6.66,6.696429,2261.646667,...,-639.298889,-958.682778,-1147.471667,3.08,2.90,2.86,2.88,3.02,2.36,1.22
2,3,53948,18.38,20.78,27.98,0.33241,2.40,7.22,6.237006,744.514444,...,-186.508889,-257.480278,-305.858056,2.44,2.42,2.42,2.44,2.46,2.46,2.02
3,4,53949,28.00,29.34,30.82,0.89333,1.34,1.50,21.126761,402.528889,...,-169.228889,-198.612222,-200.667500,1.38,1.36,1.36,1.38,1.44,1.40,1.20
4,5,53950,30.84,31.80,32.96,0.81356,0.96,1.18,28.037383,383.351111,...,-195.342222,-271.551111,-316.975556,0.98,0.98,0.96,1.00,1.12,1.02,0.82


## Extras
Clear null bytes from a file

In [ ]:
from io import open

from ventmap.clear_null_bytes import clear_descriptor_null_bytes

cleared_descriptor = clear_descriptor_null_bytes(open(<filepath to vent data>))


Cut a file into specific BN interval and store for later use

In [ ]:
from io import open

from ventmap.cut_breath_section import cut_breath_section

# get file descriptor for the truncated data
new_descriptor = cut_breath_section(open(<filepath to vent data>), <breath start num>, <breath end num>)
# write output to file
with open('new_file', 'w') as f:
    f.write(new_descriptor.read())


Check if there is a plateau pressure in a breath

In [ ]:
from io import open

from ventmap.raw_utils import extract_raw
from ventmap.SAM import check_if_plat_occurs

generator = extract_raw(open(<filepath to vent data>), False)
for breath in generator:
    flow, pressure = breath['flow'], breath['pressure']

    # .02 is the sampling rate for the PB-840 corresponding with 1 obs every .02 seconds
    did_plat_occur = check_if_plat_occurs(flow, pressure, .02)
